In [19]:
import json
from google import genai
from tqdm import tqdm
import time
client = genai.Client(api_key="GEMINI_API_KEY")


# load data/fi.json
with open("../data/formatted/fi.json", "r") as f:
    courses = json.load(f)

courses_str = [f'{course["NAME"]}\n{course["TEACHING_METHODS"]}\n{course["SYLLABUS"]}\n{course["OBJECTIVES"]}\n{course["ASSESMENT_METHODS"]}' for course in courses]
embeddings = {}
for i, course in tqdm(enumerate(courses), total=len(courses)):
    if i < 81:
        continue
    while True:
        try:
            result = client.models.embed_content(
                    model="gemini-embedding-exp-03-07",
                    contents=courses_str[i],
            )
            embeddings[course["CODE"].strip()] = result.embeddings[0].values
            break
        except Exception as e:
            print(e)
            time.sleep(30)
            continue

 22%|██▏       | 82/379 [00:00<00:02, 123.97it/s]

429 RESOURCE_EXHAUSTED. {'error': {'code': 429, 'message': 'Resource has been exhausted (e.g. check quota).', 'status': 'RESOURCE_EXHAUSTED'}}


 22%|██▏       | 82/379 [00:11<00:02, 123.97it/s]

429 RESOURCE_EXHAUSTED. {'error': {'code': 429, 'message': 'Resource has been exhausted (e.g. check quota).', 'status': 'RESOURCE_EXHAUSTED'}}


 27%|██▋       | 103/379 [01:14<04:10,  1.10it/s]

429 RESOURCE_EXHAUSTED. {'error': {'code': 429, 'message': 'Resource has been exhausted (e.g. check quota).', 'status': 'RESOURCE_EXHAUSTED'}}


 27%|██▋       | 103/379 [01:43<04:38,  1.01s/it]


KeyboardInterrupt: 

In [13]:
len(courses)

379

In [11]:
len(embeddings['CORE012'])

3072

In [ ]:
similarities = model.similarity(embeddings, embeddings)

# Print pairs with high similarity
for i in range(len(courses)):
    for j in range(i + 1, len(courses)):
        if similarities[i][j] > 0.7:
            print(f"{courses[i]['CODE']} ({courses[i]['NAME']}) - {courses[j]['CODE']} ({courses[j]['NAME']})")
            print(f"{similarities[i][j]:.2f}")
            print()

In [ ]:
# Print best pairs for course with given code
code = "PV021"
course_num = -1
for i in range(len(courses)):
    if courses[i]["CODE"].strip() == code:
        course_num = i
        break

print("Finding matches for course" + courses[course_num]["NAME"], courses[course_num]["CODE"])
print("--------------------")
best_pairs = []
pairs = [similarities[course_num][i] for i in range(len(courses))]
sorted_pairs = sorted(pairs, reverse=True)
for i in range(30):
    best_pairs.append(pairs.index(sorted_pairs[i]))

for pair in best_pairs:
    print(courses[pair]["NAME"])